In [ ]:
import json
import math

import pandas as pd
import numpy as np
import folium

In [ ]:
funding = pd.read_excel('data/ESF and ERDF operations funding - approved to May 2018.xlsx', skiprows=7, )

In [ ]:
#Drop extra columns, and space between rows
funding = funding[~funding['Programme '].isnull()]
funding = funding[funding['Operation Name'] !='Operation Name']


In [ ]:
#Add missing column names
funding = funding.rename(index=str, columns={"Categories of Intervention": "NUTS Area (Operation Only)", "Unnamed: 11": "Intervention Field 1 (ERDF & ESF)", "Unnamed: 12": "Intervention Field 2 (ERDF Only)", "Unnamed: 13": "Intervention Field 3 (ERDF Only)", "Unnamed: 14": "Form of Finance", "Unnamed: 15": "Territory Type", "Unnamed: 16": "Economic Activity"})
funding.head()

In [ ]:
funding.columns

In [ ]:
funding['postcode'] = funding['Postcode '].str.strip().str.replace(' ', '').str.replace('.', '').str.upper()

In [ ]:
postcodes = pd.read_csv('data/ukpostcodes.csv')

In [ ]:
postcodes['postcode'] = postcodes.postcode.str.strip().str.replace(' ', '')

In [ ]:
funding.merge(postcodes, left_on=['postcode'], right_on='postcode').shape

In [ ]:
funding[~funding.postcode.isin(postcodes.postcode)].shape

In [ ]:
funding[~funding.postcode.isin(postcodes.postcode)]

In [ ]:
funding = funding.merge(postcodes)
funding.head()

In [ ]:
funding['postcode'].unique().shape

In [ ]:
def make_scotland_data_geo_json(coordis_data):
    def make_feature(row):
        properties = {
            property: row[property]
            for property in ['Operation Name', 'Description', 'ESF/ ERDF Grant requested', 'Total eligible expenditure',  'Start and end dates',
                             'Investment Priority']
            if str(row[property]) != 'nan'
        }
        return {
            'type': 'Feature',
            'geometry': {
                "type": "Point",
                "coordinates": [row['latitude'], row['longitude']]
            },
            'properties': properties
        }
    features = list(coordis_data.apply(make_feature, axis=1))
    return { 'type': 'FeatureCollection', 'features': features }
with open('data/scotland_data.geo.json', 'w') as file:
    json.dump(make_scotland_data_geo_json(
        funding[~pd.isnull(funding['latitude'])]
    ), file, sort_keys=True)

mapit = folium.Map(location=[51.50810, -0.14077], zoom_start=8 )
for institute in funding.itertuples():
    if not pd.isnull(institute[20]):
        folium.Marker(location=[ institute[20], institute[21] ]).add_to(mapit)
mapit